In [2]:
import pandas as pd
import numpy as np
import time

ac = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\action_train.csv')
order = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\orderFuture_train.csv')
order_his = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\orderHistory_train.csv')
comment = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\userComment_train.csv')
profile = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\userProfile_train.csv')

ac_test = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\action_test.csv')
order_test = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\orderFuture_test.csv')
order_his_test = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\orderHistory_test.csv')
comment_test = pd.read_csv('F:\\study\\ml\\DataSet\\Hbc_csv\\userComment_test.csv')


In [3]:
ac.head()
order_his.head()

,userid,orderid,orderTime,orderType,city,country,continent
0,100000000013,1000015,1481714516,0,柏林,德国,欧洲
1,100000000013,1000014,1501959643,0,旧金山,美国,北美洲
2,100000000393,1000033,1499440296,0,巴黎,法国,欧洲
3,100000000459,1000036,1480601668,0,纽约,美国,北美洲
4,100000000459,1000034,1479146723,0,巴厘岛,印度尼西亚,亚洲


In [4]:
timeArray=time.localtime(1503443585)
time.strftime("%Y-%m-%d %H:%M:%S",timeArray)

'2017-08-23 07:13:05'

In [5]:
order[order.userid == 100000000231]  ##get row by condition
order_his.head()

,userid,orderid,orderTime,orderType,city,country,continent
0,100000000013,1000015,1481714516,0,柏林,德国,欧洲
1,100000000013,1000014,1501959643,0,旧金山,美国,北美洲
2,100000000393,1000033,1499440296,0,巴黎,法国,欧洲
3,100000000459,1000036,1480601668,0,纽约,美国,北美洲
4,100000000459,1000034,1479146723,0,巴厘岛,印度尼西亚,亚洲


In [6]:
comment.commentsKeyWords.head(10)

0                              ['很','简陋','太','随便']
1                                       ['很','细心']
2                                              NaN
3                                              NaN
4                                              NaN
5                                              NaN
6           ['真心','好','惊喜','惊喜','合理','还','真心','赞']
7                                              NaN
8    ['非常','满意','认真','负责','诚恳','合理','很','精致','希望']
9                                              NaN
Name: commentsKeyWords, dtype: object

In [7]:
def time_help(x):
    timeArray=time.localtime(x)
    return time.strftime("%Y-%m-%d %H:%M:%S",timeArray)

ac.actionTime=pd.to_datetime(ac.actionTime.apply(lambda x:time_help(x)),format='%Y-%m-%d %H:%M:%S')
order_his.orderTime = pd.to_datetime(order_his.orderTime.apply(lambda x:time_help(x)), format='%Y-%m-%d %H:%M:%S')
ac_test.actionTime = pd.to_datetime(ac_test.actionTime.apply(lambda x:time_help(x)), format='%Y-%m-%d %H:%M:%S')
order_his_test.orderTime = pd.to_datetime(order_his_test.orderTime.apply(lambda x:time_help(x)), format='%Y-%m-%d %H:%M:%S')


In [8]:
ac.head()
order_his.head()
ac_test.head()



,userid,actionType,actionTime
0,100000000371,1,2017-08-23 05:36:09
1,100000000371,5,2017-08-23 05:36:25
2,100000000371,5,2017-08-23 06:51:10
3,100000000371,5,2017-08-23 06:54:29
4,100000000371,5,2017-08-23 07:07:47


In [9]:
ac['day']=ac.actionTime.apply(lambda x:x.day)
ac_test['day']=ac.actionTime.apply(lambda x:x.day)


In [10]:
ac.head()

,userid,actionType,actionTime,day
0,100000000013,1,2016-09-19 23:59:13,19
1,100000000013,5,2016-09-19 23:59:23,19
2,100000000013,6,2016-09-20 00:01:14,20
3,100000000013,5,2016-09-20 00:01:51,20
4,100000000013,6,2016-09-20 00:02:16,20


In [15]:
def orderHistory_feat(df):
    grouped=df[['userid','orderType']].groupby('userid',as_index=False)
    df_count=grouped.count()
    df_count.rename(columns={'orderType':'df_count'},inplace=True)
    df_sum=grouped.sum()
    
    df_sum.rename(columns={'orderType':'df_sum'},inplace=True)
    df_merge=pd.merge(df_count,df_sum,on='userid',how='left')
    df_merge['rate']=df_merge['df_sum']/df_merge['df_count']
    del df_merge['df_count']
    df_merge.rename(columns={'df_sum':'orderHistory_feat_sum','rate':'orderHistory_feat_rate'},inplace=True)
    del df_merge['orderHistory_feat_rate']
    return df_merge
    

In [25]:
order_his[['userid','orderType']].groupby('userid',as_index=False).count()
orderHistory_feat(order_his).head()

,userid,orderHistory_feat_sum
0,100000000013,0
1,100000000393,0
2,100000000459,0
3,100000000637,0
4,100000000695,0


In [26]:
def get_two(arr):
    try:
        tem=int(arr.iloc[-2])
        return tem
    except:
        return np.nan

def get_tree(arr): 
    try:
        tem=int(arr.iloc[-3])
        return tem
    except:
        return np.nan



In [27]:
def last_action(df):
    last_type=df[['userid','actionType']].groupby('userid',as_index=False).agg('last')
    last_type=rename(columns={'actionType':'last_type'},inplace=True)
    
    two_type=df[['userid','actionType']].groupby('userid',as_index=False).agg(get_two)
    two_type.rename(columns={'actionType':'two_type'},inplace=True)
    
    three_type = df[['userid', 'actionType']].groupby('userid', as_index=False).agg(get_three)
    three_type.rename(columns={'actionType':'three_type'}, inplace=True)
    
    last = pd.merge(last_type, two_type, on='userid', how='left')
    last = pd.merge(last, three_type, on='userid', how='left')
    
    return last

In [52]:
ac[['userid','actionType']].groupby('userid',as_index=False).agg('last')
ac[['userid','actionType']].groupby('userid',as_index=False).agg(get_two)

ac.ix[ac.userid==100000000013]

print(ac[ac['userid']==100000000111])
ac[ac['userid']==100000000111].iloc[-2,1]
#int(ac[ac['userid']==100000000111].iloc[-2])

           userid  actionType          actionTime  day
143  100000000111           1 2017-03-31 22:43:53   31
144  100000000111           5 2017-03-31 22:44:06   31
145  100000000111           6 2017-03-31 22:44:39   31


5

In [60]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def last_city(train,test):
    city=list(order_his['city'].unique())
    city_test=list(order_his_test['city'].unique())
    for i in city_test:
        if i not in city:
            city.append(i)
    le.fit(city)
    
    new = order_his[['userid','city','continent']].drop_duplicates(subset='userid',keep='last')
    new['city']=le.transform(new['city'])
    
    new_test = order_his_test[['userid', 'city', 'continent']].drop_duplicates(subset='userid', keep='last')
    new_test['city'] = le.transform(new_test['city']) 
    
    new['continent'] = le.fit_transform(new['continent'])
    new_test['continent'] = le.transform(new_test['continent'])
    
    train = pd.merge(train, new, on='userid', how='left')
    test = pd.merge(test, new_test, on='userid', how='left')
    return train, test
        
        

In [66]:
city=list(order_his['city'].unique())
city_test=list(order_his_test['city'].unique())
for i in city_test:
    if i not in city:
        city.append(i)
le.fit(city)
le.classes_
le.transform(order_his['city'])

array([123, 116,  89, ..., 189,   2,   2], dtype=int64)

In [67]:
def profile_me(train,test):
    for i in ['province','gender','age']:
        train[i]=train[i].fac

NameError: name 'actions' is not defined